<a href="https://colab.research.google.com/github/AashiDutt/AI-and-ML-for-Coders/blob/main/Chapter4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab contains code for :
1. Fashion MNIST TensorFlow Datasets
2. Humans vs Horses TensorFlow Datasets
3. Parallelizing ETL to imporve training performance

Fashion MNIST

In [ ]:
pip install tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
(training_images, training_labels),(test_images, test_labels) = tfds.as_numpy(tfds.load('fashion_mnist', 
                                                                                        split= ['train','test'],
                                                                                        batch_size= -1,
                                                                                        as_supervised = True))


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteAXXUST/fashion_mnist-train.tfrecord


...

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/fashion_mnist/3.0.1.incompleteAXXUST/fashion_mnist-test.tfrecord


  0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset fashion_mnist downloaded and prepared to /root/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


In [ ]:
training_images = training_images/255.0
test_images = test_images/255.0


In [ ]:


model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (28,28,1)),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                                  ])
model.compile(optimizer= 'adam',
              loss= 'sparse_categorical_crossentropy',
              metrics= ['accuracy'])



In [ ]:
model.fit(training_images, training_labels, epochs = 5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5312 - accuracy: 0.8122
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3989 - accuracy: 0.8543
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3667 - accuracy: 0.8661
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3444 - accuracy: 0.8723
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3303 - accuracy: 0.8782


In [ ]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.3603 - accuracy: 0.8668


[0.360341340303421, 0.8668000102043152]

In [ ]:
classifications = model.predict(test_images)
print(classifications[0])
print(test_labels[0])

[7.3822135e-05 1.0860820e-06 3.1190047e-02 3.0384520e-06 8.8346797e-01
 2.5366245e-09 8.5223749e-02 1.2289427e-09 4.0378927e-05 7.5976542e-10]
4


Horses Vs Humans

In [2]:
pip install tensorflow-addons

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

In [ ]:
#Extract Phase
data = tfds.load("horses_or_humans", split= 'train', as_supervised = True)

#Transform Phase Start

#image augmentation with mapping function
"""def augmentimages(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/255)
  image = tf.image.random_flip_left_right(image)
  image = tfa.image.rotate(image, 40, interpolation = "NEAREST")
  return image, label

train = data.map(augmentimages)
train_batches = train.shuffle(100).batch(32)
"""
#Shufling makes training more effective
train_batches = data.shuffle(100).batch(32)

#Transform Phase End


In [ ]:
#Extract Phase
val_data = tfds.load('horses_or_humans', split = 'test', as_supervised = True)
validation_batches = val_data.batch(32)

In [ ]:
#Model Defination Start
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3), activation = 'relu', input_shape = (300,300,3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64,(3,3),activation = 'relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512,activation = 'relu'),
                                    tf.keras.layers.Dense(1, activation = 'sigmoid')
                                  ])

model.compile(optimizer = 'Adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

#Model Defination End

#Load Phase Start

history = model.fit(train_batches, epochs = 10)

#Load Phase End

Epoch 1/10
33/33 [==============================] - 6s 139ms/step - loss: 3.8522 - accuracy: 0.7955
Epoch 2/10
33/33 [==============================] - 5s 135ms/step - loss: 0.0923 - accuracy: 0.9727
Epoch 3/10
33/33 [==============================] - 5s 137ms/step - loss: 0.0350 - accuracy: 0.9903
Epoch 4/10
33/33 [==============================] - 5s 136ms/step - loss: 0.0068 - accuracy: 0.9990
Epoch 5/10
33/33 [==============================] - 5s 135ms/step - loss: 7.7286e-04 - accuracy: 1.0000
Epoch 6/10
33/33 [==============================] - 5s 135ms/step - loss: 0.0187 - accuracy: 0.9922
Epoch 7/10
33/33 [==============================] - 5s 134ms/step - loss: 0.2150 - accuracy: 0.9484
Epoch 8/10
33/33 [==============================] - 5s 134ms/step - loss: 0.0804 - accuracy: 0.9688
Epoch 9/10
33/33 [==============================] - 5s 138ms/step - loss: 0.0430 - accuracy: 0.9864
Epoch 10/10
33/33 [==============================] - 5s 135ms/step - loss: 0.0288 - accuracy: 0.

In [ ]:
history = model.fit(train_batches, epochs = 10, validation_data= validation_batches, validation_steps = 1)

Epoch 1/10
33/33 [==============================] - 6s 151ms/step - loss: 0.0026 - accuracy: 0.9990 - val_loss: 0.5993 - val_accuracy: 0.9375
Epoch 2/10
33/33 [==============================] - 5s 140ms/step - loss: 4.0252e-04 - accuracy: 1.0000 - val_loss: 0.5965 - val_accuracy: 0.9688
Epoch 3/10
33/33 [==============================] - 5s 142ms/step - loss: 1.2165e-04 - accuracy: 1.0000 - val_loss: 0.6068 - val_accuracy: 0.9688
Epoch 4/10
33/33 [==============================] - 5s 141ms/step - loss: 7.2305e-05 - accuracy: 1.0000 - val_loss: 0.6496 - val_accuracy: 0.9688
Epoch 5/10
33/33 [==============================] - 5s 141ms/step - loss: 5.4086e-05 - accuracy: 1.0000 - val_loss: 0.6788 - val_accuracy: 0.9688
Epoch 6/10
33/33 [==============================] - 5s 144ms/step - loss: 4.1603e-05 - accuracy: 1.0000 - val_loss: 0.7049 - val_accuracy: 0.9375
Epoch 7/10
33/33 [==============================] - 5s 142ms/step - loss: 3.4423e-05 - accuracy: 1.0000 - val_loss: 0.7246 - val

Parallelizing ETL to improve training performance

In [4]:
#Get training data(Extract)
train_data = tfds.load('cats_vs_dogs', split = 'train', with_info=True)

#Use tfRecord to access Raw files that were downloaded
file_pattern = f'/root/tensorflow_datasets/cats_vs_dogs/4.0.0/cats_vs_dogs-train.tfrecord*'
files = tf.data.Dataset.list_files(file_pattern)

In [5]:
# These files are loaded into dataset using files.interleave
train_dataset = files.interleave(tf.data.TFRecordDataset,
                                 cycle_length = 4, #number of input elemments processed concurrently
                                 num_parallel_calls = tf.data.experimental.AUTOTUNE) #no. of parallel calls to execute

In [6]:
import multiprocessing

In [7]:
#Transform Phase
#create mapping fun that loads raw TFRecord and converts it to usable content

def read_tfrecord(serialized_example):
  feature_description = {
      "image":tf.io.FixedLenFeature((),tf.string,""),
      "label":tf.io.FixedLenFeature((),tf.int64,-1),
  }
  example = tf.io.parse_single_example(
       serialized_example, feature_description
  )
  image = tf.io.decode_jpeg(example['image'],channels = 3)
  image = tf.cast(image, tf.float32)
  image = image / 255
  image = tf.image.resize(image,(224,224))
  return image, example['label']

#calling mapping function

cores = multiprocessing.cpu_count()


print(cores)
train_dataset = train_dataset.map(read_tfrecord, num_parallel_calls= cores)
train_dataset = train_dataset.cache()



2


In [8]:
#Parallelizing Batching and shuffling

train_dataset = train_dataset.shuffle(1024).batch(8) #16/8/32
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

#prefetching decreases the idle time of CPU and GPU


In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', 
                                      input_shape=(224, 224, 3)), #224 or 300
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

model.compile(optimizer = 'Adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])


In [ ]:
model.fit(train_dataset, epochs = 10, verbose = 1)

Epoch 1/10
   2238/Unknown - 874s 389ms/step - loss: 0.6937 - accuracy: 0.4982

In [ ]:
#Runtime crashed